# Explore `EncQA` data 

In [39]:
import polars as pl
import os 
from PIL import Image 
import io 
import base64
import json 

In [7]:
os.getcwd()

'/Users/shenglong/Downloads/ml-encqa'

In [9]:
df = pl.read_parquet("data/encqa_v1.parquet")
df.head()

image_path,question,true_label,options,task,task_details,encoding,variable_type,answer_type,num_marks,num_categories,chart_spec,image,split,canary_guid
str,str,str,list[str],str,str,str,str,str,i8,i8,str,struct[2],str,str
"""synthetic_data/images/find_ext…","""which color has the most obser…","""Green""","[""Blue"", ""Purple"", … ""Orange""]","""find_extrema""","""{""extremum"": ""maximum""}""","""color_nominal""","""nominal""","""multiple_choice""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\x81IDATx\x9c\xed\xddoL\x1c\xf7\x99\xc0\xf1\xb1Ycf\xd7\x10\xb3\x0b""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/find_ext…","""which color has the least obse…","""Blue""","[""Red"", ""Green"", … ""Orange""]","""find_extrema""","""{""extremum"": ""minimum""}""","""color_nominal""","""nominal""","""multiple_choice""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\x81IDATx\x9c\xed\xddoL\x1c\xf7\x99\xc0\xf1\xb1Ycf\xd7\x10\xb3\x0b""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/find_ext…","""which color has the most obser…","""Red""","[""Blue"", ""Purple"", … ""Orange""]","""find_extrema""","""{""extremum"": ""maximum""}""","""color_nominal""","""nominal""","""multiple_choice""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x170IDATx\x9c\xed\xddol\xd3\xf7\x9d\xc0\xf1o\xfe`\xf89\x7f\xc0q""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/find_ext…","""which color has the least obse…","""Purple""","[""Green"", ""Red"", … ""Purple""]","""find_extrema""","""{""extremum"": ""minimum""}""","""color_nominal""","""nominal""","""multiple_choice""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x170IDATx\x9c\xed\xddol\xd3\xf7\x9d\xc0\xf1o\xfe`\xf89\x7f\xc0q""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/find_ext…","""which color has the most obser…","""Purple""","[""Orange"", ""Blue"", … ""Purple""]","""find_extrema""","""{""extremum"": ""maximum""}""","""color_nominal""","""nominal""","""multiple_choice""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\xf9IDATx\x9c\xed\xddMl\x9b\xf7}\xc0\xf1\xc7\xa6D\x89\x94HK""""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"


In [15]:
df.group_by("task").len()

task,len
str,u32
"""compute_derived_value_exact""",275
"""compute_derived_value_relative""",225
"""correlate_values""",300
"""correlate_values_relative""",150
"""retrieve_value""",225
"""find_anomaly""",450
"""filter_values""",450
"""find_extrema""",450


In [22]:
ret_df = df.filter(pl.col("task").is_in(["retrieve_value", "compute_derived_value_exact"]))
ret_df.head()

image_path,question,true_label,options,task,task_details,encoding,variable_type,answer_type,num_marks,num_categories,chart_spec,image,split,canary_guid
str,str,str,list[str],str,str,str,str,str,i8,i8,str,struct[2],str,str
"""synthetic_data/images/retrieve…","""How many Purple circles are pr…","""2""",null,"""retrieve_value""","""{""target_category"": ""Purple""}""","""color_nominal""","""nominal""","""numeric""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\x81IDATx\x9c\xed\xddoL\x1c\xf7\x99\xc0\xf1\xb1Ycf\xd7\x10\xb3\x0b""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""How many Red circles are prese…","""2""",null,"""retrieve_value""","""{""target_category"": ""Red""}""","""color_nominal""","""nominal""","""numeric""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\xbdIDATx\x9c\xed\xdd\x7fL\x9b\xf7\x9d\xc0\xf1o\x02\x18\x8c1\x891\x04""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""How many Blue circles are pres…","""1""",null,"""retrieve_value""","""{""target_category"": ""Blue""}""","""color_nominal""","""nominal""","""numeric""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\x11IDATx\x9c\xed\xdd]lT\xe7\x99\xc0\xf1\x87x<x<\xb6\xc3\xf8""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""How many Blue circles are pres…","""2""",null,"""retrieve_value""","""{""target_category"": ""Blue""}""","""color_nominal""","""nominal""","""numeric""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\xc6IDATx\x9c\xed\xddml\x1b\xf7}\xc0\xf1\xbf\x9e(\x1f)\xd9\x92(""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""How many Green circles are pre…","""1""",null,"""retrieve_value""","""{""target_category"": ""Green""}""","""color_nominal""","""nominal""","""numeric""",12,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xa0\x00\x00\x01\xa0\x08\x02\x00\x00\x00cT$\x0e\x00\x00\x16\xa4IDATx\x9c\xed\xddol\xd3\xf7\x9d\xc0\xf1\x0f$1\xd8N\x00\xc7\xf9""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"


In [23]:
ret_df.group_by("encoding").len()

encoding,len
str,u32
"""area""",100
"""color_quantitative""",50
"""length""",100
"""color_nominal""",75
"""position""",100
"""shape""",75


In [24]:
ret_df = ret_df.filter(pl.col("encoding").is_in(["position", "length"]))
ret_df.group_by("encoding").len()

encoding,len
str,u32
"""position""",100
"""length""",100


In [27]:
ret_df.head()

image_path,question,true_label,options,task,task_details,encoding,variable_type,answer_type,num_marks,num_categories,chart_spec,image,split,canary_guid
str,str,str,list[str],str,str,str,str,str,i8,i8,str,struct[2],str,str
"""synthetic_data/images/retrieve…","""What is the value of Var at A?""","""38.88""",null,"""retrieve_value""","""{""target_category"": ""A""}""","""position""","""quantitative""","""numeric""",5,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xc1\x00\x00\x01\xb0\x08\x02\x00\x00\x00^\xa7I\xdb\x00\x00\x1b\x15IDATx\x9c\xed\xdd\xbfo\xdbH\xc2\xf0\xf1\xf1\x8b\x05,\xe0)4\xa9""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""What is the value of Var at E?""","""53.38""",null,"""retrieve_value""","""{""target_category"": ""E""}""","""position""","""quantitative""","""numeric""",5,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xaa\x00\x00\x01\xba\x08\x02\x00\x00\x00\xd6;\x95:\x00\x00\x16\xdeIDATx\x9c\xed\xdd\xbfs\xdaj\xbe\x80q\xe5Nf\xd0\xcc\x16h+""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""What is the value of Var at C?""","""120.93""",null,"""retrieve_value""","""{""target_category"": ""C""}""","""position""","""quantitative""","""numeric""",5,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xc1\x00\x00\x01\xb0\x08\x02\x00\x00\x00^\xa7I\xdb\x00\x00\x1b\xc2IDATx\x9c\xed\xdd?l\xdbH\xc2\xb0q\xfa\xc3\x02\x16\xf0\x16\x1eW""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""What is the value of Var at C?""","""120.93""",null,"""retrieve_value""","""{""target_category"": ""C""}""","""position""","""quantitative""","""numeric""",5,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xaa\x00\x00\x01\xba\x08\x02\x00\x00\x00\xd6;\x95:\x00\x00\x17\xf4IDATx\x9c\xed\xdd\xbfs\xdaj\xbe\x80q\xe5Nf`f\x0bt*""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"
"""synthetic_data/images/retrieve…","""What is the value of Var at B?""","""46.5""",null,"""retrieve_value""","""{""target_category"": ""B""}""","""position""","""quantitative""","""numeric""",5,5,"""{""config"": {""view"": {""continuo…","{b""\x89PNG\x0d\x0a\x1a\x0a\x00\x00\x00\x0dIHDR\x00\x00\x01\xbc\x00\x00\x01\xb0\x08\x02\x00\x00\x00]\x10G\xf3\x00\x00\x1aSIDATx\x9c\xed\xdd\xbfo\xdbH\xe2\xb0\xf1\xf1\x8b\x05,\xe0\x0a\xd1U""…,null}","""synthetic_data""","""826ba6c6-5e2d-4b59-8684-77e497…"


We will save these images somewhere, so we can exmaine them further: 

In [28]:
def render_image(image_bytes):
    return Image.open(io.BytesIO(base64.b64decode(image_bytes))) 
    
    

In [43]:
for i, row in enumerate(ret_df.head(5).iter_rows(named=True)): 
    json_data = json.loads(row['chart_spec'])
    with open(f"data/img/{i}.json", "w") as f: 
        # json.dump(row['chart_spec'], f)
        # f.write(json_data)
        json.dump(json_data, f, indent=4, ensure_ascii=False)
    
    
    